In [2]:
import pandas as pd
import numpy as np
import os
from transformers import BertTokenizer, BertForSequenceClassification, DistilBertTokenizer, DistilBertForSequenceClassification, RobertaTokenizer, RobertaForSequenceClassification
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from sklearn.metrics import classification_report, accuracy_score
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
from transformers import RobertaTokenizer, RobertaForSequenceClassification
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pandas as pd
from transformers import (BertTokenizerFast, BertForSequenceClassification,
                          DistilBertTokenizerFast, DistilBertForSequenceClassification,
                          RobertaTokenizerFast, RobertaForSequenceClassification,
                          Trainer, TrainingArguments)
from nltk.sentiment import SentimentIntensityAnalyzer
import nltk

# Tải từ điển cảm xúc của Vader (nếu chưa tải)
nltk.download('vader_lexicon')


c:\Users\Admin\miniconda3\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Error loading vader_lexicon: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>


False

### LOAD DATASET


In [3]:

# Load data
data = pd.read_csv('comments_labeled.csv')
data

,id,date,user_id,user_username,user_displayname,replyCount,retweetCount,likeCount,quoteCount,viewCount,searched_keyword,cleaned_text,Sentiment,hashtag_text
0,1.850000e+18,2024-11-05 14:23:43+00:00,2.237591e+09,ImKnotTheOne,Charli Huxley,169,326,2788,33,158104,VoteTrump VoteKamala ElectionDay Election2024 ...,Because only garbage people would vote for thi...,Negative,"Trump2024, VotedforTrump, ElectionDay"
1,1.850000e+18,2024-10-23 15:22:44+00:00,5.377095e+08,LauraLoomer,Laura Loomer,2205,8598,47970,176,550747,VoteTrump VoteKamala ElectionDay Election2024 ...,JUST VOTED IN FLORIDA FOR THE GREATEST PRESIDE...,Positive,"Trump2024, AmericaFirst"
2,1.850000e+18,2024-11-05 11:34:56+00:00,2.237591e+09,ImKnotTheOne,Charli Huxley,190,735,6803,44,346181,VoteTrump VoteKamala ElectionDay Election2024 ...,Vote as if your life depends on it. and FCK Do...,Negative,"ElectionDay, Election2024, VotedforTrump"
3,1.850000e+18,2024-11-05 15:39:26+00:00,4.729379e+07,w_terrence,Terrence K Williams,265,1252,6069,31,67609,VoteTrump VoteKamala ElectionDay Election2024 ...,TODAY IS THAT DAY Lets do this. Get up and go ...,Negative,"ElectionDay, electionday2024"
4,1.840000e+18,2024-09-19 20:23:25+00:00,1.360000e+18,kimmagagal2,Kimmie,31,243,600,8,19341,VoteTrump VoteKamala ElectionDay Election2024 ...,A message for Kamaaaala voters! Do you want pe...,Negative,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1161,1.850000e+18,2024-10-13 03:21:37+00:00,1.370000e+18,IRAtheGhostARI,IRA The Ghost of ARI u O,114,182,1459,14,57632,Election2024 Trump 2024 VoteForTrump VoteTrump...,realDonaldTrump If you are voting TRUMP hit th...,Positive,NaN
1162,1.850000e+18,2024-10-17 16:28:08+00:00,1.520000e+18,MarindaVannoy1,Mandy,137,131,1612,10,21270,Election2024 Trump 2024 VoteForTrump VoteTrump...,VOTE FIGHT VOTE FIGHT VOTE FIGHT VOTE FIGHT VO...,Negative,NaN
1163,1.850000e+18,2024-11-04 16:42:28+00:00,1.450000e+18,PURE_BL00D2,S I G,25,263,289,5,10903,Election2024 Trump 2024 VoteForTrump VoteTrump...,IT'S OUR TIME PATRIOTS..! History in the makin...,Positive,"TrumpVance2024, Vote"
1164,1.850000e+18,2024-11-05 15:08:33+00:00,1.430000e+18,Sassafrass_84,Sassafrass84,22,170,508,7,6842,Election2024 Trump 2024 VoteForTrump VoteTrump...,Today is the day. Don't sit this one out. Go v...,Positive,NaN


### Chuyển 3 sentiment Positive, Negative , Neutral thành dạng số
+ clean lại một lần nữa các ký tự đặc biệt


In [4]:

# Cleaning the text data (removing unwanted characters, unnecessary spaces)
data['cleaned_text'] = data['cleaned_text'].str.replace('[^a-zA-Z0-9\s]', '', regex=True)

# Mapping sentiment labels to numerical values for training models
sentiment_map = {'Positive': 1, 'Negative': 0, 'Neutral': 2}
data['Sentiment_num'] = data['Sentiment'].map(sentiment_map)


<>:2: SyntaxWarning: invalid escape sequence '\s'
<>:2: SyntaxWarning: invalid escape sequence '\s'
C:\Users\Admin\AppData\Local\Temp\ipykernel_6204\2522857093.py:2: SyntaxWarning: invalid escape sequence '\s'
  data['cleaned_text'] = data['cleaned_text'].str.replace('[^a-zA-Z0-9\s]', '', regex=True)


### chia train, test + khởi tạo các tokenizers


In [5]:

# Splitting data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(data['cleaned_text'], data['Sentiment_num'], test_size=0.2, random_state=42)

# Initializing tokenizers for BERT, RoBERTa, and DistilBERT
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
roberta_tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
distilbert_tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')


c:\Users\Admin\miniconda3\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


### Training model BERT


In [6]:
import torch
from sklearn.metrics import classification_report, accuracy_score
from transformers import BertForSequenceClassification, BertTokenizer, RobertaForSequenceClassification, RobertaTokenizer, DistilBertForSequenceClassification, DistilBertTokenizer
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm

# Function to tokenize data for transformer models
def tokenize_data(tokenizer, texts, max_length=64):
    return tokenizer(texts.tolist(), truncation=True, padding=True, max_length=max_length, return_tensors='pt')

# Initialize Tokenizers for BERT, RoBERTa, and DistilBERT
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
roberta_tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
distilbert_tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

# Function to create DataLoader for batch training
def create_data_loader(input_ids, attention_mask, labels, batch_size=16):
    dataset = TensorDataset(input_ids, attention_mask, labels)
    return DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Define the model training function
def train_model(model, train_loader, optimizer, epochs=3):
    model.train()
    for epoch in range(epochs):
        epoch_loss = 0
        for batch in tqdm(train_loader, desc=f"Epoch {epoch+1}"):
            optimizer.zero_grad()
            input_ids, attention_mask, labels = batch
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()
        print(f'Epoch {epoch+1} - Average Loss: {epoch_loss / len(train_loader)}')

# Function to evaluate the model
def evaluate_model(model, test_loader):
    model.eval()
    predictions, true_labels = [], []
    with torch.no_grad():
        for batch in tqdm(test_loader, desc="Evaluating"):
            input_ids, attention_mask, labels = batch
            outputs = model(input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            _, predicted = torch.max(logits, dim=1)
            predictions.extend(predicted.cpu().numpy())
            true_labels.extend(labels.cpu().numpy())
    return predictions, true_labels

# Create train and test data loaders for BERT
train_bert_tokens = tokenize_data(bert_tokenizer, X_train)
test_bert_tokens = tokenize_data(bert_tokenizer, X_test)

train_bert_loader = create_data_loader(train_bert_tokens['input_ids'], train_bert_tokens['attention_mask'], torch.tensor(y_train.tolist()))
test_bert_loader = create_data_loader(test_bert_tokens['input_ids'], test_bert_tokens['attention_mask'], torch.tensor(y_test.tolist()))

# Initialize BERT model and optimizer
bert_model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=3)
optimizer = torch.optim.Adam(bert_model.parameters(), lr=1e-5)

# Train and evaluate BERT model
train_model(bert_model, train_bert_loader, optimizer)
predictions, true_labels = evaluate_model(bert_model, test_bert_loader)
print("BERT Classification Report:")
print(classification_report(true_labels, predictions))


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Epoch 1: 100%|██████████| 59/59 [01:31<00:00,  1.55s/it]


Epoch 1 - Average Loss: 0.9892058059320612


Epoch 2: 100%|██████████| 59/59 [01:25<00:00,  1.44s/it]


Epoch 2 - Average Loss: 0.7974704352475829


Epoch 3: 100%|██████████| 59/59 [01:23<00:00,  1.42s/it]


Epoch 3 - Average Loss: 0.5364569299301859


Evaluating: 100%|██████████| 15/15 [00:05<00:00,  2.59it/s]

BERT Classification Report:
              precision    recall  f1-score   support

           0       0.78      0.81      0.79        77
           1       0.83      0.80      0.82       122
           2       0.62      0.66      0.64        35

    accuracy                           0.78       234
   macro avg       0.75      0.76      0.75       234
weighted avg       0.78      0.78      0.78       234



### Traning RoBERTa, DistilBERT

In [7]:
# Create train and test data loaders for RoBERTa
train_roberta_tokens = tokenize_data(roberta_tokenizer, X_train)
test_roberta_tokens = tokenize_data(roberta_tokenizer, X_test)

train_roberta_loader = create_data_loader(train_roberta_tokens['input_ids'], train_roberta_tokens['attention_mask'], torch.tensor(y_train.tolist()))
test_roberta_loader = create_data_loader(test_roberta_tokens['input_ids'], test_roberta_tokens['attention_mask'], torch.tensor(y_test.tolist()))

# Initialize RoBERTa model and optimizer
roberta_model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=3)
optimizer_roberta = torch.optim.Adam(roberta_model.parameters(), lr=1e-5)

# Train and evaluate RoBERTa model
print("Training RoBERTa Model...")
train_model(roberta_model, train_roberta_loader, optimizer_roberta)
predictions_roberta, true_labels_roberta = evaluate_model(roberta_model, test_roberta_loader)
print("RoBERTa Classification Report:")
print(classification_report(true_labels_roberta, predictions_roberta))

# Create train and test data loaders for DistilBERT
train_distilbert_tokens = tokenize_data(distilbert_tokenizer, X_train)
test_distilbert_tokens = tokenize_data(distilbert_tokenizer, X_test)

train_distilbert_loader = create_data_loader(train_distilbert_tokens['input_ids'], train_distilbert_tokens['attention_mask'], torch.tensor(y_train.tolist()))
test_distilbert_loader = create_data_loader(test_distilbert_tokens['input_ids'], test_distilbert_tokens['attention_mask'], torch.tensor(y_test.tolist()))

# Initialize DistilBERT model and optimizer
distilbert_model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=3)
optimizer_distilbert = torch.optim.Adam(distilbert_model.parameters(), lr=1e-5)

# Train and evaluate DistilBERT model
print("Training DistilBERT Model...")
train_model(distilbert_model, train_distilbert_loader, optimizer_distilbert)
predictions_distilbert, true_labels_distilbert = evaluate_model(distilbert_model, test_distilbert_loader)
print("DistilBERT Classification Report:")
print(classification_report(true_labels_distilbert, predictions_distilbert))


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Training RoBERTa Model...


Epoch 1: 100%|██████████| 59/59 [01:25<00:00,  1.45s/it]


Epoch 1 - Average Loss: 1.0001868171206976


Epoch 2: 100%|██████████| 59/59 [01:25<00:00,  1.45s/it]


...

Epoch 2 - Average Loss: 0.7943821574671793


Epoch 3: 100%|██████████| 59/59 [00:43<00:00,  1.36it/s]


Epoch 3 - Average Loss: 0.6143481559672598


Evaluating: 100%|██████████| 15/15 [00:03<00:00,  4.82it/s]

DistilBERT Classification Report:
              precision    recall  f1-score   support

           0       0.78      0.82      0.80        77
           1       0.78      0.86      0.82       122
           2       0.63      0.34      0.44        35

    accuracy                           0.77       234
   macro avg       0.73      0.67      0.69       234
weighted avg       0.76      0.77      0.76       234



### Training model Logistic regression and Vader

In [8]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import numpy as np

# Logistic Regression with TF-IDF
def train_logistic_regression(X_train, y_train, X_test, y_test):
    tfidf_vectorizer = TfidfVectorizer(stop_words='english', max_features=5000)
    X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
    X_test_tfidf = tfidf_vectorizer.transform(X_test)
    
    # Train Logistic Regression
    logreg_model = LogisticRegression(max_iter=1000)
    logreg_model.fit(X_train_tfidf, y_train)
    
    # Evaluate the model
    logreg_preds = logreg_model.predict(X_test_tfidf)
    print("Logistic Regression Classification Report:")
    print(classification_report(y_test, logreg_preds))
    
    return logreg_model, tfidf_vectorizer

# Vader Sentiment Analysis
def evaluate_vader(X_test, y_test):
    analyzer = SentimentIntensityAnalyzer()
    
    vader_preds = []
    for text in X_test:
        score = analyzer.polarity_scores(text)['compound']
        if score > 0.05:
            vader_preds.append(1)  # Positive
        elif score < -0.05:
            vader_preds.append(0)  # Negative
        else:
            vader_preds.append(2)  # Neutral
    
    print("Vader Sentiment Classification Report:")
    print(classification_report(y_test, vader_preds))

# Huấn luyện Logistic Regression
logreg_model, tfidf_vectorizer = train_logistic_regression(X_train, y_train, X_test, y_test)

# Đánh giá với Vader Sentiment
evaluate_vader(X_test, y_test)


Logistic Regression Classification Report:
              precision    recall  f1-score   support

           0       0.71      0.68      0.69        77
           1       0.71      0.84      0.77       122
           2       0.81      0.37      0.51        35

    accuracy                           0.72       234
   macro avg       0.75      0.63      0.66       234
weighted avg       0.73      0.72      0.71       234

Vader Sentiment Classification Report:
              precision    recall  f1-score   support

           0       0.75      0.53      0.62        77
           1       0.67      0.75      0.71       122
           2       0.26      0.31      0.29        35

    accuracy                           0.62       234
   macro avg       0.56      0.53      0.54       234
weighted avg       0.63      0.62      0.62       234



# Save the trained model with mlflow

In [13]:
import mlflow
import mlflow.pyfunc
import torch
from transformers import BertTokenizer, BertForSequenceClassification, RobertaTokenizer, RobertaForSequenceClassification, DistilBertTokenizer, DistilBertForSequenceClassification

mlflow.set_experiment("sentiment-analysis")

# --- Custom Wrapper for HuggingFace Transformers ---
class HFTransformersWrapper(mlflow.pyfunc.PythonModel):
    def __init__(self, model_class, tokenizer_class, model_path):
        self.model_class = model_class
        self.tokenizer_class = tokenizer_class
        self.model_path = model_path

    def load_context(self, context):
        self.model = self.model_class.from_pretrained(self.model_path)
        self.tokenizer = self.tokenizer_class.from_pretrained(self.model_path)

    def predict(self, context, model_input):
        texts = model_input["text"].tolist()
        inputs = self.tokenizer(texts, padding=True, truncation=True, max_length=64, return_tensors="pt")
        with torch.no_grad():
            outputs = self.model(**inputs)
            preds = torch.argmax(outputs.logits, dim=1).numpy()
        return preds

# --- Save BERT model ---
bert_model.save_pretrained("bert_saved")
bert_tokenizer.save_pretrained("bert_saved")
with mlflow.start_run(run_name="bert_sentiment"):
    mlflow.pyfunc.log_model(
        "bert_model",
        python_model=HFTransformersWrapper(BertForSequenceClassification, BertTokenizer, "bert_saved"),
        input_example=pd.DataFrame({"text": X_test.tolist()[:2]})
    )
    mlflow.log_param("model_type", "BERT")
    acc = accuracy_score(true_labels, predictions)
    report = classification_report(true_labels, predictions, output_dict=True)
    mlflow.log_metric("accuracy", acc)
    mlflow.log_metric("precision", report["weighted avg"]["precision"])
    mlflow.log_metric("recall", report["weighted avg"]["recall"])
    mlflow.log_metric("f1_score", report["weighted avg"]["f1-score"])
    print("BERT model saved to MLflow.")

# --- Save RoBERTa model ---
roberta_model.save_pretrained("roberta_saved")
roberta_tokenizer.save_pretrained("roberta_saved")
with mlflow.start_run(run_name="roberta_sentiment"):
    mlflow.pyfunc.log_model(
        "roberta_model",
        python_model=HFTransformersWrapper(RobertaForSequenceClassification, RobertaTokenizer, "roberta_saved"),
        input_example=pd.DataFrame({"text": X_test.tolist()[:2]})
    )
    mlflow.log_param("model_type", "RoBERTa")
    acc = accuracy_score(true_labels_roberta, predictions_roberta)
    report = classification_report(true_labels_roberta, predictions_roberta, output_dict=True)
    mlflow.log_metric("accuracy", acc)
    mlflow.log_metric("precision", report["weighted avg"]["precision"])
    mlflow.log_metric("recall", report["weighted avg"]["recall"])
    mlflow.log_metric("f1_score", report["weighted avg"]["f1-score"])
    print("RoBERTa model saved to MLflow.")

# --- Save DistilBERT model ---
distilbert_model.save_pretrained("distilbert_saved")
distilbert_tokenizer.save_pretrained("distilbert_saved")
with mlflow.start_run(run_name="distilbert_sentiment"):
    mlflow.pyfunc.log_model(
        "distilbert_model",
        python_model=HFTransformersWrapper(DistilBertForSequenceClassification, DistilBertTokenizer, "distilbert_saved"),
        input_example=pd.DataFrame({"text": X_test.tolist()[:2]})
    )
    mlflow.log_param("model_type", "DistilBERT")
    acc = accuracy_score(true_labels_distilbert, predictions_distilbert)
    report = classification_report(true_labels_distilbert, predictions_distilbert, output_dict=True)
    mlflow.log_metric("accuracy", acc)
    mlflow.log_metric("precision", report["weighted avg"]["precision"])
    mlflow.log_metric("recall", report["weighted avg"]["recall"])
    mlflow.log_metric("f1_score", report["weighted avg"]["f1-score"])
    print("DistilBERT model saved to MLflow.")

# Save Logistic Regression model with TF-IDF
class SklearnTextWrapper(mlflow.pyfunc.PythonModel):
    def __init__(self, model, vectorizer):
        self.model = model
        self.vectorizer = vectorizer

    def predict(self, context, model_input):
        X = self.vectorizer.transform(model_input["text"])
        return self.model.predict(X)

# Evaluate the Logistic Regression model
logreg_preds = logreg_model.predict(tfidf_vectorizer.transform(X_test))
logreg_report = classification_report(y_test, logreg_preds, output_dict=True)
logreg_acc = accuracy_score(y_test, logreg_preds)

with mlflow.start_run(run_name="logreg_sentiment_pyfunc"):
    mlflow.pyfunc.log_model(
        "logreg_model",
        python_model=SklearnTextWrapper(logreg_model, tfidf_vectorizer),
        input_example=pd.DataFrame({"text": X_test.tolist()[:2]})
    )
    mlflow.log_param("model_type", "LogisticRegression")
    mlflow.log_metric("accuracy", logreg_acc)
    mlflow.log_metric("precision", logreg_report["weighted avg"]["precision"])
    mlflow.log_metric("recall", logreg_report["weighted avg"]["recall"])
    mlflow.log_metric("f1_score", logreg_report["weighted avg"]["f1-score"])
    print("Logistic Regression pyfunc model saved to MLflow.")

# --- Save Vader Sentiment as a custom pyfunc model 
class VaderSentimentWrapper(mlflow.pyfunc.PythonModel):
    def __init__(self):
        self.analyzer = SentimentIntensityAnalyzer()
    def predict(self, context, model_input):
        results = []
        for text in model_input["text"]:
            score = self.analyzer.polarity_scores(text)["compound"]
            if score > 0.05:
                results.append(1)
            elif score < -0.05:
                results.append(0)
            else:
                results.append(2)
        return results

vader_model = VaderSentimentWrapper()
vader_preds = vader_model.predict(None, pd.DataFrame({"text": X_test.tolist()}))
vader_report = classification_report(y_test, vader_preds, output_dict=True)
with mlflow.start_run(run_name="vader_sentiment"):
    mlflow.pyfunc.log_model("vader_model", python_model=vader_model, input_example=pd.DataFrame({"text": X_test.tolist()[:2]}))
    mlflow.log_param("model_type", "VaderSentiment")
    mlflow.log_metric("accuracy", accuracy_score(y_test, vader_preds))
    mlflow.log_metric("precision", vader_report["weighted avg"]["precision"])
    mlflow.log_metric("recall", vader_report["weighted avg"]["recall"])
    mlflow.log_metric("f1_score", vader_report["weighted avg"]["f1-score"])
    print("Vader Sentiment model saved to MLflow.")

c:\Users\Admin\miniconda3\Lib\site-packages\mlflow\pyfunc\utils\data_validation.py:186: UserWarning: Add type hints to the `predict` method to enable data validation and automatic signature inference during model logging. Check https://mlflow.org/docs/latest/model/python_model.html#type-hint-usage-in-pythonmodel for more details.
  color_warning(
2025/04/21 13:04:13 INFO mlflow.pyfunc: Inferring model signature from input example
c:\Users\Admin\miniconda3\Lib\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns ma

BERT model saved to MLflow.


2025/04/21 13:04:24 INFO mlflow.pyfunc: Inferring model signature from input example
c:\Users\Admin\miniconda3\Lib\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(


RoBERTa model saved to MLflow.


2025/04/21 13:04:35 INFO mlflow.pyfunc: Inferring model signature from input example
c:\Users\Admin\miniconda3\Lib\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(
c:\Users\Admin\miniconda3\Lib\site-packages\mlflow\pyfunc\utils\data_validation.py:186: UserWarning: Add type hints to the `predict` method to enable data vali

DistilBERT model saved to MLflow.


c:\Users\Admin\miniconda3\Lib\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(
c:\Users\Admin\miniconda3\Lib\site-packages\mlflow\pyfunc\utils\data_validation.py:186: UserWarning: Add type hints to the `predict` method to enable data validation and automatic signature inference during model logging. Check https://mlflow.o

Logistic Regression pyfunc model saved to MLflow.


c:\Users\Admin\miniconda3\Lib\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(

Vader Sentiment model saved to MLflow.


In [15]:
import mlflow
print(mlflow.get_tracking_uri())

file:///d:/python/MLOps/project/mlruns


# Register

In [16]:
mlflow.register_model("runs:/e578ccd39bcd45e8a31e8a4e0adc8afa/bert_model", "SentimentBERT")
mlflow.register_model("runs:/fc2e60ac916a40b79bbb3b0466de979e/roberta_model", "SentimentRoBERTa")
mlflow.register_model("runs:/c20fe5721d8b44c69d7eb1f8e827a69d/distilbert_model", "SentimentDistilBERT")
mlflow.register_model("runs:/e17c8020f9f0402eb8b8dc72cb33dbb8/logreg_model", "SentimentLogReg")
mlflow.register_model("runs:/bebfdc14154646c3b0312818e82ff464/vader_model", "SentimentVader")

Registered model 'SentimentBERT' already exists. Creating a new version of this model...
Created version '3' of model 'SentimentBERT'.
Registered model 'SentimentRoBERTa' already exists. Creating a new version of this model...
Created version '3' of model 'SentimentRoBERTa'.
Registered model 'SentimentDistilBERT' already exists. Creating a new version of this model...
Created version '3' of model 'SentimentDistilBERT'.
Registered model 'SentimentLogReg' already exists. Creating a new version of this model...
Created version '3' of model 'SentimentLogReg'.
Registered model 'SentimentVader' already exists. Creating a new version of this model...
Created version '3' of model 'SentimentVader'.


<ModelVersion: aliases=[], creation_timestamp=1745215699812, current_stage='None', description=None, last_updated_timestamp=1745215699812, name='SentimentVader', run_id='bebfdc14154646c3b0312818e82ff464', run_link=None, source='file:///d:/python/MLOps/project/mlruns/759481644003022958/bebfdc14154646c3b0312818e82ff464/artifacts/vader_model', status='READY', status_message=None, tags={}, user_id=None, version=3>

In [17]:
from mlflow import MlflowClient 
client = MlflowClient()

# Define the model names and their respective performance metrics
model_performance = {
    "SentimentBERT": {
        "accuracy": accuracy_score(true_labels, predictions),
        "precision": report["weighted avg"]["precision"],
        "recall": report["weighted avg"]["recall"],
        "f1_score": report["weighted avg"]["f1-score"]
    },
    "SentimentRoBERTa": {
        "accuracy": accuracy_score(true_labels_roberta, predictions_roberta),
        "precision": classification_report(true_labels_roberta, predictions_roberta, output_dict=True)["weighted avg"]["precision"],
        "recall": classification_report(true_labels_roberta, predictions_roberta, output_dict=True)["weighted avg"]["recall"],
        "f1_score": classification_report(true_labels_roberta, predictions_roberta, output_dict=True)["weighted avg"]["f1-score"]
    },
    "SentimentDistilBERT": {
        "accuracy": accuracy_score(true_labels_distilbert, predictions_distilbert),
        "precision": classification_report(true_labels_distilbert, predictions_distilbert, output_dict=True)["weighted avg"]["precision"],
        "recall": classification_report(true_labels_distilbert, predictions_distilbert, output_dict=True)["weighted avg"]["recall"],
        "f1_score": classification_report(true_labels_distilbert, predictions_distilbert, output_dict=True)["weighted avg"]["f1-score"]
    },
    "SentimentLogReg": {
        "accuracy": logreg_acc,
        "precision": report["weighted avg"]["precision"],
        "recall": report["weighted avg"]["recall"],
        "f1_score": report["weighted avg"]["f1-score"]
    },
    "SentimentVader": {
        "accuracy": accuracy_score(y_test, vader_preds),
        "precision": vader_report["weighted avg"]["precision"],
        "recall": vader_report["weighted avg"]["recall"],
        "f1_score": vader_report["weighted avg"]["f1-score"]
    }
}

# Sort models by a primary metric (F1-score) in descending order
sorted_models = sorted(model_performance.items(), key=lambda x: x[1]["f1_score"], reverse=True)

# Assign champion and challenger based on the primary metric
models = {
    sorted_models[0][0]: {"production": 1, "alias_champion": "champion"},
    sorted_models[1][0]: {"staging": 2, "alias_challenger": "challenger"}
}

# Update stages and set aliases for champion and challenger
for model_name, details in models.items():
    if "production" in details:
        client.transition_model_version_stage(
            name=model_name,
            version=details["production"],
            stage="Production"
        )
        print(f"Model {model_name} version {details['production']} set to Production.")

        client.set_registered_model_alias(
            name=model_name,
            alias=details["alias_champion"],
            version=details["production"]
        )
        print(f"Alias '{details['alias_champion']}' set for model {model_name} version {details['production']}.")

    if "staging" in details:
        client.transition_model_version_stage(
            name=model_name,
            version=details["staging"],
            stage="Staging"
        )
        print(f"Model {model_name} version {details['staging']} set to Staging.")

        client.set_registered_model_alias(
            name=model_name,
            alias=details["alias_challenger"],
            version=details["staging"]
        )
        print(f"Alias '{details['alias_challenger']}' set for model {model_name} version {details['staging']}.")

# Retrieve and print the champion and challenger models
for model_name in models.keys():
    if "alias_champion" in models[model_name]:
        champion_version = client.get_model_version_by_alias(name=model_name, alias="champion").version
        print(f"Model {model_name}: Champion version is {champion_version}.")
    if "alias_challenger" in models[model_name]:
        challenger_version = client.get_model_version_by_alias(name=model_name, alias="challenger").version
        print(f"Model {model_name}: Challenger version is {challenger_version}.")

Model SentimentRoBERTa version 1 set to Production.
Alias 'champion' set for model SentimentRoBERTa version 1.
Model SentimentBERT version 2 set to Staging.
Alias 'challenger' set for model SentimentBERT version 2.
Model SentimentRoBERTa: Champion version is 1.
Model SentimentBERT: Challenger version is 2.


C:\Users\Admin\AppData\Local\Temp\ipykernel_6204\2278271366.py:50: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(
C:\Users\Admin\AppData\Local\Temp\ipykernel_6204\2278271366.py:65: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


In [18]:
# Retrieve run_id of champion
champion_model_version = client.get_model_version_by_alias(name="SentimentRoBERTa", alias="champion")
champion_run_id = champion_model_version.run_id
print(f"Champion Run ID: {champion_run_id}")

# Retrieve run_id of challenger
challenger_model_version = client.get_model_version_by_alias(name="SentimentBERT", alias="challenger")
challenger_run_id = challenger_model_version.run_id
print(f"Challenger Run ID: {challenger_run_id}")

Champion Run ID: fc2e60ac916a40b79bbb3b0466de979e
Challenger Run ID: e578ccd39bcd45e8a31e8a4e0adc8afa


# Load model

In [30]:
import mlflow.pyfunc

# Load champion model (example: SentimentRoBERTa)
champion_model = mlflow.pyfunc.load_model(model_uri="models:/SentimentRoBERTa@champion")

# Load challenger model (example: SentimentBERT)
challenger_model = mlflow.pyfunc.load_model(model_uri="models:/SentimentBERT@challenger")

# Example prediction
import pandas as pd
df = pd.DataFrame({"text": ["donal trump is stupid"]})

print("Champion prediction:", champion_model.predict(df))
print("Challenger prediction:", challenger_model.predict(df))

Champion prediction: [0]
Challenger prediction: [0]


# Serve the model via MLflow's endpoint (run this in terminal, not notebook)
- mlflow models serve -m "models:/SentimentRoBERTa@champion" -p 5001 --no-conda


In [29]:
import requests
import json

url = "http://127.0.0.1:5001/invocations"  
data = {
  "inputs": [
    {"text": "donal trump is a bit wierd, but still okay"},
  ]
}
response = requests.post(url, data=json.dumps(data), headers={"Content-Type": "application/json"})
print(response.json())

{'predictions': [2]}
